# Chargement données d'entrainement

In [114]:
# Chargement de la librairie
from fastai2.vision.all import *

# Récupération des images
path = Path('/storage/data/mnist_sample')
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
three_tensor = [tensor(Image.open(o)) for o in threes]
seven_tensor = [tensor(Image.open(o)) for o in sevens]

# On ramène les valeurs entre 0 et 255 à leur équivalent entre 0 et 1. 
# On simplifie la liste pour obtenir une liste ??
stacked_threes = torch.stack(three_tensor).float()/255
stacked_sevens = torch.stack(seven_tensor).float()/255

# On regroupe ces informations dans un seul tensor
# On crée également un tensor avec les valeur attendues (1 c'est un trois, 0 ce n'est pas un trois)
stacked_digits = torch.cat([stacked_threes, stacked_sevens])
stacked_labels = tensor([1]*len(threes) + [0]*len(sevens))

# On transforme les tableaux de 28 par 28 pixel en une liste de 784 pixels pour simplifier le traitement 
# On formate les deux tensors pour être équivalent 
train_x = stacked_digits.view(-1, 28*28)
train_y = stacked_labels.unsqueeze(1)
train_dataset = list(zip(train_x,train_y))

# On met en batch 
train_dl = DataLoader(train_dataset,batch_size=256)

# Chargement données de validation

In [115]:
# On charge les images valides. 
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255

# On concatène et crée la liste de label
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dataset = list(zip(valid_x,valid_y))

#On met en batch
valid_dl = DataLoader(valid_dataset, batch_size=256)

# Définition des méthodes 

### Initialisation des paramètres

In [116]:
# On rends un tensor de la taille demandée avec des paramètres définis au hasard. 
# On positionne la propriété requires_grad pour pouvoir calculer le gradient par la suite.
def init_params(size, std=1.0):
    return (torch.randn(size)*std).requires_grad_()

### Fonction de calcul de coût (loss function)

In [117]:
# On utilise le sigmoid pour ramener le résultat obtenu par le model à sa valeur entre 0 et 1
# On renvoit la moyenne des distances d'erreur
def mnist_loss(activations_01, targets):
        activations_01 = activations_01.sigmoid()
        distances = torch.where(targets==1,1-activations_01,activations_01)
        return distances.mean()

### Calcul de la performance 

In [118]:
# On calcul le nombre de prédictions correctes 
def accuracy(activations, targets):
    predictions = activations > 0
    corrects = (predictions == targets)
    return corrects.float().mean()

### Calcul du gradient 

In [144]:
# On applique le model puis on appel backward (propagation) pour calculer l'ensemble des gradients
def compute_gradient(xb, yb, model, loss_func): 
    predictions = model(xb)
    loss = loss_func(predictions, yb)
    loss.backward()

### Définition du model

In [120]:
# On utilise un modèle linéaire ici avec simplement la multiplication de chaque pixel par le paramètre correspondant. 
# On a 784 pixel et 784 paramètres. Petit à petit les paramètres en face de pixel hautement probable pour un trois vont 
# être augmenter et inversement. 
# params[0] contient les poids et params[1] le biais (pour stabiliser ?)
def linear_model(xb):
    return xb@weights + bias

# Définition des méthodes époques

In [145]:
# On parcours les mini-batch et on va calculer les gradients puis appliquer la learning rate sur chaque paramètre
def train_epoch(model, lr, params):
    for xb, yb in train_dl:
        compute_gradient(xb,yb,model,mnist_loss)
        for p in params:
            p.data -= p.grad * lr # On modifie chaque paramètre en fonction du gradient et du learning rate défini
            p.grad.zero_() # remise à zéros du gradient pour itération suivante. 

# On parcours les mini batchs du dataset de validation et on calcule le taux de réussite avec le model actuel. 
# On retourne la moyenne de ces taux de réussite
def validate_epoch(model):
        accs = [accuracy(model(xb), yb) for xb,yb in valid_dl]
        return round(torch.stack(accs).mean().item(), 4)

# On entraine le model

In [146]:
lr = 1
weights = init_params((28*28,1))
bias = init_params(1)
params = (weights,bias)

for i in range(20):
    train_epoch(linear_model, lr, params)
    print(validate_epoch(linear_model), end=' ')

0.5853 0.8007 0.891 0.9203 0.9398 0.9472 0.9516 0.954 0.956 0.9584 0.9594 0.9613 0.9628 0.9638 0.9647 0.9657 0.9657 0.9667 0.9677 0.9687 